# Cats and Dogs Image CNN

In [17]:
import tensorflow as tf
 
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Conv2D, Flatten, Dropout, MaxPooling2D, Activation
from tensorflow.keras.preprocessing.image import ImageDataGenerator
 
import os
import numpy as np
import matplotlib.pyplot as plt

# dimensions of our images.
img_width, img_height = 150, 150

train_data_dir = 'data/train'
validation_data_dir = 'data/validation'
nb_train_samples = 2000
nb_validation_samples = 800
epochs = 10
batch_size = 16

# build CNN model
model = Sequential()
model.add(Conv2D(32, (3, 3), input_shape=(img_width, img_height, 3)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Conv2D(32, (3, 3)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Conv2D(64, (3, 3)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Flatten())
model.add(Dense(64))
model.add(Activation('relu'))
model.add(Dropout(0.5))
model.add(Dense(2))
model.add(Activation('sigmoid'))
model.summary()

Model: "sequential_5"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_15 (Conv2D)           (None, 148, 148, 32)      896       
_________________________________________________________________
activation_20 (Activation)   (None, 148, 148, 32)      0         
_________________________________________________________________
max_pooling2d_14 (MaxPooling (None, 74, 74, 32)        0         
_________________________________________________________________
conv2d_16 (Conv2D)           (None, 72, 72, 32)        9248      
_________________________________________________________________
activation_21 (Activation)   (None, 72, 72, 32)        0         
_________________________________________________________________
max_pooling2d_15 (MaxPooling (None, 36, 36, 32)        0         
_________________________________________________________________
conv2d_17 (Conv2D)           (None, 34, 34, 64)       

In [18]:
# compile model
model.compile(loss='categorical_crossentropy',
              optimizer='rmsprop',
              metrics=['accuracy'])

In [19]:
# this is the augmentation configuration we will use for training
train_datagen = ImageDataGenerator(
    rescale=1. / 255,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True)

# this is the augmentation configuration we will use for testing:
# only rescaling
test_datagen = ImageDataGenerator(rescale=1. / 255)


# class_mode = "binary" if # classes = 2 
# class_mode = "categorical" if # classes >= 2
train_generator = train_datagen.flow_from_directory(
    train_data_dir,
    target_size=(img_width, img_height),
    batch_size=batch_size,
    class_mode='categorical')

validation_generator = test_datagen.flow_from_directory(
    validation_data_dir,
    target_size=(img_width, img_height),
    batch_size=batch_size,
    class_mode='categorical')

Found 2000 images belonging to 2 classes.
Found 800 images belonging to 2 classes.


In [20]:
model.fit_generator(
    train_generator,
    steps_per_epoch=nb_train_samples // batch_size,
    epochs=epochs,
    validation_data=validation_generator,
    validation_steps=nb_validation_samples // batch_size)

Instructions for updating:
Please use Model.fit, which supports generators.
Epoch 1/10
125/125 [==============================] - 26s 206ms/step - loss: 0.7042 - accuracy: 0.5135 - val_loss: 0.6703 - val_accuracy: 0.5400
Epoch 2/10
125/125 [==============================] - 20s 161ms/step - loss: 0.6651 - accuracy: 0.6090 - val_loss: 0.6413 - val_accuracy: 0.6200
Epoch 3/10
125/125 [==============================] - 19s 153ms/step - loss: 0.6293 - accuracy: 0.6590 - val_loss: 0.6689 - val_accuracy: 0.6137
Epoch 4/10
125/125 [==============================] - 19s 152ms/step - loss: 0.6191 - accuracy: 0.6540 - val_loss: 0.6197 - val_accuracy: 0.6787
Epoch 5/10
125/125 [==============================] - 19s 155ms/step - loss: 0.5973 - accuracy: 0.6950 - val_loss: 0.5993 - val_accuracy: 0.6650
Epoch 6/10
125/125 [==============================] - 19s 152ms/step - loss: 0.5755 - accuracy: 0.7020 - val_loss: 0.7708 - val_accuracy: 0.5962
Epoch 7/10
125/125 [==============================] - 

In [21]:
model.save('CNN_model.h5')

In [22]:
import tensorflow as tf

for cls, idx in train_generator.class_indices.items():
    print('Class #{} = {}'.format(idx, cls))
    
test_image = tf.keras.preprocessing.image.load_img('data/validation/dogs/dog.1010.jpg', target_size = (150, 150)) 
test_image = tf.keras.preprocessing.image.img_to_array(test_image)
test_image = test_image /255.0
test_image = np.expand_dims(test_image, axis = 0)

#predict the result
result = model.predict(test_image)
print(result)

Class #0 = cats
Class #1 = dogs
[[0.01402488 0.9976629 ]]
